<a href="https://colab.research.google.com/github/btcnhung1299/tf-practice/blob/master/TXT_SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import tensorflow as tf
import numpy as np

## Data gathering

In [2]:
(ds_train, ds_val), ds_info = tfds.load("imdb_reviews", split=["train", "test"], shuffle_files=True, as_supervised=True, with_info=True)
ds_info

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete6R0WAF/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete6R0WAF/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete6R0WAF/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
      title     = {Learning Word

**Inspect samples from datasets**

Binary classification:
- Negative: 0
- Positive: 1

In [3]:
sample = next(iter(ds_train))
print("Review:", sample[0].numpy())
print("Label:", sample[1].numpy())

Review: b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no explanation of what was causing them or why. I admit, I may have missed part of the film, but i watched the majority of it and everything just seemed to happen of its own accord without any real concern for anything else. I cant recommend this film at all.'
Label: 0


## Featurizing

In [4]:
train_reviews, train_labels = [], []
for review, label in ds_train:
  train_reviews.append(str(review.numpy()))
  train_labels.append(label.numpy())

val_reviews, val_labels = [], []
for review, label in ds_val:
  val_reviews.append(str(review.numpy()))
  val_labels.append(label.numpy())

print("Number of training samples:", len(train_reviews))
print("Number of validation samples:", len(val_reviews))

Number of training samples: 25000
Number of validation samples: 25000


**Tokenizing and padding sequences**

In [5]:
BATCH_SIZE = 64
VOCAB_SIZE = 8000
EMBED_DIM = 64
MAX_SEQ_LEN = 256

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>")
tokenizer.fit_on_texts(train_reviews)

In [7]:
def create_tfds(tokenizer, X, y):
  seq_X = pad_sequences(tokenizer.texts_to_sequences(X), maxlen=MAX_SEQ_LEN, padding="post", truncating="post")
  return tf.data.Dataset.from_tensor_slices((seq_X, y)).shuffle(1024).batch(BATCH_SIZE)

ds_train = create_tfds(tokenizer, train_reviews, train_labels)
ds_val = create_tfds(tokenizer, val_reviews, val_labels)

## Model Architecture

### Without transfer learning

In [33]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, Bidirectional, LSTM, GRU, Dropout, GlobalAveragePooling1D

In [37]:
model = Sequential()
model.add(Embedding(input_dim=VOCAB_SIZE, input_length=MAX_SEQ_LEN, output_dim=EMBED_DIM))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(64, activation="swish"))
model.add(Dense(1, activation="sigmoid"))
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 256, 64)           512000    
_________________________________________________________________
bidirectional_5 (Bidirection (None, 128)               66048     
_________________________________________________________________
dense_6 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 65        
Total params: 586,369
Trainable params: 586,369
Non-trainable params: 0
_________________________________________________________________


In [38]:
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["acc"])

In [39]:
history = model.fit(ds_train, epochs=10, validation_data=ds_val)

Epoch 1/10
391/391 [==============================] - 15s 39ms/step - loss: 0.4930 - acc: 0.7732 - val_loss: 0.4973 - val_acc: 0.7991
Epoch 2/10
391/391 [==============================] - 14s 37ms/step - loss: 0.3180 - acc: 0.8744 - val_loss: 0.4099 - val_acc: 0.8106
Epoch 3/10
391/391 [==============================] - 15s 38ms/step - loss: 0.2655 - acc: 0.8993 - val_loss: 0.3598 - val_acc: 0.8404
Epoch 4/10
391/391 [==============================] - 14s 37ms/step - loss: 0.2373 - acc: 0.9111 - val_loss: 0.3992 - val_acc: 0.8185
Epoch 5/10
391/391 [==============================] - 14s 37ms/step - loss: 0.2150 - acc: 0.9228 - val_loss: 0.3634 - val_acc: 0.8475
Epoch 6/10
391/391 [==============================] - 15s 37ms/step - loss: 0.1985 - acc: 0.9290 - val_loss: 0.3913 - val_acc: 0.8508
Epoch 7/10
391/391 [==============================] - 14s 37ms/step - loss: 0.1808 - acc: 0.9350 - val_loss: 0.4235 - val_acc: 0.8558
Epoch 8/10
391/391 [==============================] - 14s 37ms